# FastText

A diferencia de Word2Vec, que trabaja a nivel de palabra, FastText trata de capturar la información morfológica de las palabras.

>*"[...] we propose a new approach **based on the skipgram model, where each word is represented as a bag of character n-grams**. A vector representation is associated to each character n-gram; words being represented as the sum of these representations. [...]"* <br>(Mikolov et al., Enriching Word Vectors with Subword Information, https://arxiv.org/pdf/1607.04606.pdf)

De esta manera, una palabra quedará representada por sus n-grams.

El tamaño de los n-grams deberá ser definido como hiperparámetro
- min_n: valor mínimo de _n_ a considerar
- max_n: valor máximo de _n_ a considerar

Ejemplo:
>*"Me gusta el procesado del lenguaje natural"*
>* Ejemplo de *skip-gram* pre-procesado con una ventana de contexto de 2 palabras
>
>$w_{target} =$ "procesado" &emsp;$w_{context} =$ ["gusta", "el", "del", "lenguaje"] 
>
>     ("procesado", "gusta")
>
> Descomoposición de n-grams con min_n=3 and max_n=4:
>
>"procesado" = ["$<$pr", "pro", ..., "ado", "do$>$", "$<$pro", "roce", ..., "sado", "ado$>$"]
>
>* De este modo, la similitud será: <br><br>
>&emsp;$\boxed{s(w_{target}, w_{context}) = \sum_{g \in G_{w_{target}}}z_{g}^T v_{w_{context}}}$, where $G_{w_{target}}\subset\{g_{1}, ..., g_{G}\}$

## Palabras más similares

In [1]:
import numpy as np

In [2]:
def print_sim_words(word, model1, model2):
    query = "Most similar to {}".format(word) 
    print(query)
    print("-"*len(query))
    for (sim1, sim2) in zip(model1.wv.most_similar(word), model2.wv.most_similar(word)):
        print("{}:{}{:.3f}{}{}:{}{:.3f}".format(sim1[0],
                                               " "*(20-len(sim1[0])), 
                                               sim1[1], 
                                               " "*10, 
                                               sim2[0],
                                               " "*(20-len(sim2[0])),
                                               sim2[1]))
    print("\n")

## Importamos las librerías

In [3]:
from gensim.models import FastText
from gensim.models.word2vec import LineSentence

## Lectura de datos

In [4]:
corpus = LineSentence('../../datasets/spanish_news_corpus_doc.txt', limit=250)

## Hyperparameters

In [5]:
sg_params = {
    'sg': 1,
    'size': 300,
    'min_count': 5,
    'window': 5,
    'hs': 0,
    'negative': 20,
    'workers': 4,
    'min_n': 3,
    'max_n': 6
}

cbow_params = {
    'sg': 0,
    'size': 300,
    'min_count': 5,
    'window': 5,
    'hs': 0,
    'negative': 20,
    'workers': 4,
    'min_n': 3,
    'max_n': 6
}

## Inicializamos el objeto FastText

In [6]:
# Skip Gram
ft_sg = FastText(**sg_params)

# CBOW
ft_cbow = FastText(**cbow_params)

## Construímos el vocabulario

In [7]:
# Skip Gram
ft_sg.build_vocab(corpus)

# CBOW
ft_cbow.build_vocab(corpus)

In [8]:
print('Vocabulario compuesto por {} palabras'.format(len(ft_sg.wv.vocab)))

Vocabulario compuesto por 3139 palabras


In [9]:
print('Vocabulario compuesto por {} palabras'.format(len(ft_cbow.wv.vocab)))

Vocabulario compuesto por 3139 palabras


## Entrenamos los pesos de los embeddings

In [10]:
# Skip Gram
ft_sg.train(sentences=corpus, total_examples=ft_sg.corpus_count, epochs=20)

In [11]:
# CBOW
ft_cbow.train(sentences=corpus, total_examples=ft_cbow.corpus_count, epochs=20)

## Guardamos los modelos

In [12]:
ft_sg.save('../../data/ft_sg_d300_mc5_w5.pkl')
ft_cbow.save('../../data/ft_cbow_d300_mc5_w5.pkl')

## Algunos resultados

In [13]:
print_sim_words('elecciones', ft_cbow, ft_sg)
print_sim_words('botín', ft_cbow, ft_sg)
print_sim_words('sánchez', ft_cbow, ft_sg)
print_sim_words('rafa', ft_cbow, ft_sg)
print_sim_words('impeachment', ft_cbow, ft_sg)

Most similar to elecciones
--------------------------
funciones:           0.990          funciones:           0.848
sanciones:           0.987          posiciones:          0.848
condiciones:         0.980          subvenciones:        0.832
subvenciones:        0.975          acciones:            0.815
acciones:            0.974          reuniones:           0.812
vacaciones:          0.971          vacaciones:          0.808
negociaciones:       0.971          delegaciones:        0.807
situaciones:         0.969          cuestiones:          0.804
conversaciones:      0.968          sanciones:           0.799
soluciones:          0.968          conclusiones:        0.789


Most similar to botín
---------------------
álvarez:             0.988          álvarez:             0.868
archidiócesis:       0.983          elche:               0.850
lópez:               0.982          palomera:            0.842
igae:                0.978          arena:               0.823
champions:        

## Out-of-Vocabulary (OOV) Words 

la cantidad de n-grams creados durante el entrenamiento del FastText hace improbable (que no imposible) que alguna palabra no pueda ser construída como una bolsa de n-grams

In [14]:
'asereje' in ft_sg.wv.vocab

False

In [15]:
ft_sg.wv.most_similar('asereje')

[('vilamitjana', 0.8189496994018555),
 ('blog', 0.8152294754981995),
 ('adquirir', 0.8094627857208252),
 ('bitcoin', 0.8001219034194946),
 ('antigua', 0.7970510721206665),
 ('envió', 0.7890565395355225),
 ('streaming', 0.7890428304672241),
 ('enorme', 0.7873703241348267),
 ('discurso', 0.7858148813247681),
 ('moverse', 0.78006511926651)]

In [35]:
ft_sg.wv['asereje'].shape

(300,)